In [8]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler

# Sample data
data = {
    'Date': ['2020-01-01', '2020-01-02', '2020-01-03', '2020-01-04', '2020-01-05'],
    'Sales': [33, 15, 24, 38, 29],
    'DayOfWeek': [2, 3, 4, 5, 6],
    'Weekend': [0, 0, 0, 1, 1]
}

df = pd.DataFrame(data)

# Check if there is enough data for the sequence length
sequence_length = 4
if len(df) <= sequence_length:
    raise ValueError("Not enough data to create sequences with the given sequence length.")

# Preparing sequences
X, y = [], []
for i in range(len(df) - sequence_length):
    X.append(df[['Sales', 'DayOfWeek', 'Weekend']].iloc[i:i + sequence_length].values)
    y.append(df[['Sales', 'DayOfWeek', 'Weekend']].iloc[i + sequence_length].values)

X = np.array(X)
y = np.array(y)

# Scaling the data
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X.reshape(-1, X.shape[-1])).reshape(X.shape)
y_scaled = scaler.transform(y)

# Reshape for LSTM
X_scaled = X_scaled.reshape((X_scaled.shape[0], X_scaled.shape[1], X_scaled.shape[2]))
y_scaled = y_scaled.reshape((y_scaled.shape[0], y_scaled.shape[1]))

# Define the LSTM model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(X_scaled.shape[1], X_scaled.shape[2])))
model.add(Dense(y_scaled.shape[1]))
model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(X_scaled, y_scaled, epochs=300, verbose=1)

# Prepare the input for the next prediction
last_sequence = df[['Sales', 'DayOfWeek', 'Weekend']].iloc[-sequence_length:].values
last_sequence_scaled = scaler.transform(last_sequence)
last_sequence_scaled = last_sequence_scaled.reshape((1, last_sequence_scaled.shape[0], last_sequence_scaled.shape[1]))

# Predict the next values
predicted_scaled = model.predict(last_sequence_scaled)
predicted = scaler.inverse_transform(predicted_scaled)

# Extract the predicted Sales, DayOfWeek, and Weekend
predicted_sales = predicted[0, 0]
predicted_day_of_week = int(predicted[0, 1])
predicted_weekend = int(predicted[0, 2])

predicted_sales, predicted_day_of_week, predicted_weekend


Epoch 1/300
1/1 [==============================] - 2s 2s/step - loss: 1.0580
Epoch 2/300
1/1 [==============================] - 0s 12ms/step - loss: 1.0433
Epoch 3/300
1/1 [==============================] - 0s 13ms/step - loss: 1.0288
Epoch 4/300
1/1 [==============================] - 0s 17ms/step - loss: 1.0144
Epoch 5/300
1/1 [==============================] - 0s 12ms/step - loss: 1.0003
Epoch 6/300
1/1 [==============================] - 0s 14ms/step - loss: 0.9865
Epoch 7/300
1/1 [==============================] - 0s 13ms/step - loss: 0.9729
Epoch 8/300
1/1 [==============================] - 0s 13ms/step - loss: 0.9597
Epoch 9/300
1/1 [==============================] - 0s 17ms/step - loss: 0.9467
Epoch 10/300
1/1 [==============================] - 0s 11ms/step - loss: 0.9337
Epoch 11/300
1/1 [==============================] - 0s 12ms/step - loss: 0.9211
Epoch 12/300
1/1 [==============================] - 0s 12ms/step - loss: 0.9086
Epoch 13/300
1/1 [==============================] -

(34.887115, 7, 1)